# リカレントニューラルネットワーク (RNN: LSTM)
時系列データや文章などの系列データ（Sequence Data）を扱うためのニューラルネットワーク  
MLPやCNNとは異なり、過去の情報を保持しながら処理を行うため、時系列データの依存関係を学習するのに適している
## RNNの基礎概念
時系列ごとにデータを処理し、過去の情報を現在の計算に活用。  
内部状態（隠れ状態）を持ち、系列のつながりを考慮できる。
## RNNの課題
短期の依存関係は学習できるが、長期の依存関係は苦手。  
勾配消失問題（Vanishing Gradient Problem） により、長い系列では過去の情報が伝わりにくい。
→LSTMやGRUが開発された
## LSTM(Long Short-Term Memory）とは？
RNN の勾配消失問題を解決するために設計されたモデル  
ゲート機構を導入することで、長期依存関係を効率的に学習できるようになっている
## LSTMの内部構造
### 入力ゲート
新しい情報をどの程度メモリに保存するかを決定
### 忘却ゲート
過去の情報をどの程度忘れるかを決定
### 出力ゲート
次の隠れ状態の値をどの程度出力するかを決定
## メリット
長期依存関係を学習できる  
勾配消失問題を軽減  
時系列データ（株価、気象データなど）や自然言語処理（NLP）に強い  
## デメリット
計算コストが高い（通常の RNN より遅い）  
単純な短期依存関係なら GRU（Gated Recurrent Unit）の方が効率的

In [2]:
import torch
import torch.nn as nn

class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super(LSTMClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, (h_n, c_n) = self.lstm(embedded)
        out = self.fc(h_n[-1])
        return out


In [ ]:

import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
from torch.nn.utils.rnn import pad_sequence


# ダミーの IMDB レビュー（単語 ID のリスト）
dummy_train_data = [
    (torch.randint(0, 5000, (200,)), 0),  # シーケンス長200, 0:ネガティブ
    (torch.randint(0, 5000, (150,)), 1),  # シーケンス長150, 1:ポジティブ
]
dummy_test_data = [
    (torch.randint(0, 5000, (200,)), 1),
    (torch.randint(0, 5000, (100,)), 0),
]

class IMDBDataset(Dataset):
    def __init__(self, data_list):
        self.data_list = data_list  # [(encoded_text, label), ...]

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        return self.data_list[idx]

def collate_fn(batch):
    texts, labels = zip(*batch)  # [(text1, label1), (text2, label2), ...] → ([text1, text2], [label1, label2])

    # **パディングを適用**
    padded_texts = pad_sequence(texts, batch_first=True, padding_value=0)  # 0 でパディング
    labels = torch.tensor(labels, dtype=torch.long)  # ラベルを Tensor に変換

    return padded_texts, labels

train_dataset = IMDBDataset(dummy_train_data)
test_dataset = IMDBDataset(dummy_test_data)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False, collate_fn=collate_fn)


model = LSTMClassifier(vocab_size=5000, embed_dim=128, hidden_dim=64, output_dim=2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for texts, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(texts)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")



Epoch 1/3, Loss: 0.6913
Epoch 2/3, Loss: 0.5883
Epoch 3/3, Loss: 0.4985
